## Descrizione del dataset

Il dataset contiene le registrazioni dei viaggi in taxi effettuati a New York City in un periodo che va dal INIZIO al FINE.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn as sk
from sklearn.preprocessing import MinMaxScaler

taxi_df = pd.read_csv('/content/drive/MyDrive/data.csv')
taxi_df.head()

<ipython-input-2-75fe026fe5e4>:9: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  taxi_df = pd.read_csv('/content/drive/MyDrive/data.csv')


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.2,1.0,N,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.2,1.0,N,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.6,1.0,N,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.8,1.0,N,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0


In [3]:
print(taxi_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6405008 entries, 0 to 6405007
Data columns (total 18 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VendorID               float64
 1   tpep_pickup_datetime   object 
 2   tpep_dropoff_datetime  object 
 3   passenger_count        float64
 4   trip_distance          float64
 5   RatecodeID             float64
 6   store_and_fwd_flag     object 
 7   PULocationID           int64  
 8   DOLocationID           int64  
 9   payment_type           float64
 10  fare_amount            float64
 11  extra                  float64
 12  mta_tax                float64
 13  tip_amount             float64
 14  tolls_amount           float64
 15  improvement_surcharge  float64
 16  total_amount           float64
 17  congestion_surcharge   float64
dtypes: float64(13), int64(2), object(3)
memory usage: 879.6+ MB
None


In [4]:
taxi_df.isnull().sum()
#taxi_df.count()
num_records = len(taxi_df)
print(f"Number of records in the dataframe: {num_records}")

Number of records in the dataframe: 6405008


Sostituisco valori nulli

In [5]:
taxi_df['payment_type'] = taxi_df['payment_type'].fillna(5.0)
taxi_df['RatecodeID'] = taxi_df['RatecodeID'].fillna(99.0)
taxi_df['VendorID'] = taxi_df['VendorID'].fillna(99.0)

In [6]:
taxi_df.drop(columns=['store_and_fwd_flag'], inplace=True)

In [7]:
taxi_df = taxi_df[(taxi_df['passenger_count'] > 0) &
              (taxi_df['trip_distance'] >= 0) &
              (taxi_df['fare_amount'] >= 0) &
              (taxi_df['extra'] >= 0) &
              (taxi_df['mta_tax'] >= 0) &
              (taxi_df['tip_amount'] >= 0) &
              (taxi_df['tolls_amount'] >= 0) &
              (taxi_df['improvement_surcharge'] >= 0) &
              (taxi_df['congestion_surcharge'] >= 0) &
               (taxi_df['tpep_dropoff_datetime'] > taxi_df['tpep_pickup_datetime']) ].copy()

taxi_df.isnull().sum()

,0
VendorID,0
tpep_pickup_datetime,0
tpep_dropoff_datetime,0
passenger_count,0
trip_distance,0
RatecodeID,0
PULocationID,0
DOLocationID,0
payment_type,0
fare_amount,0


In [8]:
taxi_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2020-01-01 00:28:15,2020-01-01 00:33:03,1.0,1.20,1.0,238,239,1.0,6.0,3.0,0.5,1.47,0.0,0.3,11.27,2.5
1,1.0,2020-01-01 00:35:39,2020-01-01 00:43:04,1.0,1.20,1.0,239,238,1.0,7.0,3.0,0.5,1.50,0.0,0.3,12.30,2.5
2,1.0,2020-01-01 00:47:41,2020-01-01 00:53:52,1.0,0.60,1.0,238,238,1.0,6.0,3.0,0.5,1.00,0.0,0.3,10.80,2.5
3,1.0,2020-01-01 00:55:23,2020-01-01 01:00:14,1.0,0.80,1.0,238,151,1.0,5.5,0.5,0.5,1.36,0.0,0.3,8.16,0.0
4,2.0,2020-01-01 00:01:58,2020-01-01 00:04:16,1.0,0.00,1.0,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,4.80,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6339562,2.0,2020-01-31 23:38:07,2020-01-31 23:52:21,1.0,2.10,1.0,163,246,1.0,11.0,0.5,0.5,2.96,0.0,0.3,17.76,2.5
6339563,2.0,2020-01-31 23:00:18,2020-01-31 23:19:18,1.0,2.13,1.0,164,79,1.0,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5
6339564,2.0,2020-01-31 23:24:22,2020-01-31 23:40:39,1.0,2.55,1.0,79,68,1.0,12.5,0.5,0.5,3.26,0.0,0.3,19.56,2.5
6339565,2.0,2020-01-31 23:44:22,2020-01-31 23:54:00,1.0,1.61,1.0,100,142,2.0,8.5,0.5,0.5,0.00,0.0,0.3,12.30,2.5


Agggiungiamo colonne di interesse

In [9]:
# Convert pickup and dropoff columns to datetime
taxi_df['tpep_pickup_datetime'] = pd.to_datetime(taxi_df['tpep_pickup_datetime'])
taxi_df['tpep_dropoff_datetime'] = pd.to_datetime(taxi_df['tpep_dropoff_datetime'])
# Calculate trip duration in minutes
taxi_df['trip_duration'] = (taxi_df['tpep_dropoff_datetime'] - taxi_df['tpep_pickup_datetime']).dt.total_seconds() / 60
# Calculating fare per mile for a better analysis
taxi_df['fare_per_mile'] = np.where(taxi_df['trip_distance'] != 0, taxi_df['total_amount'] / taxi_df['trip_distance'], 0)

Impostare categorie

In [10]:
# Convert specific columns to categorical
taxi_df['VendorID'] = taxi_df['VendorID'].astype('category')
taxi_df['payment_type'] = taxi_df['payment_type'].astype('category')
taxi_df['RatecodeID'] = taxi_df['RatecodeID'].astype('category')

In [11]:


taxi_df.iloc[200]

,201
VendorID,2.0
tpep_pickup_datetime,2020-01-01 00:53:40
tpep_dropoff_datetime,2020-01-01 01:08:02
passenger_count,1.0
trip_distance,2.32
RatecodeID,1.0
PULocationID,114
DOLocationID,45
payment_type,1.0
fare_amount,11.0


## Dataset Describe

In [12]:
num_records = len(taxi_df)
print(f"Number of records in the dataframe: {num_records}")

# Calcolo dei quartili
trip_distance_Q1 = taxi_df['trip_distance'].quantile(0.25)
trip_distance_Q3 = taxi_df['trip_distance'].quantile(0.75)
trip_distance_IQR = trip_distance_Q3 - trip_distance_Q1

# Soglie per identificare gli outlier
trip_distance_lower_bound = trip_distance_Q1 - 1.5 * trip_distance_IQR
trip_distance_upper_bound = trip_distance_Q3 + 1.5 * trip_distance_IQR

print(f"Soglia inferiore trip_distance: {trip_distance_lower_bound}")
print(f"Soglia superiore trip_distance: {trip_distance_upper_bound}")

# Calcolo dei quartili
trip_duration_Q1 = taxi_df['trip_duration'].quantile(0.25)
trip_duration_Q3 = taxi_df['trip_duration'].quantile(0.75)
trip_duration_IQR = trip_duration_Q3 - trip_duration_Q1

# Soglie per identificare gli outlier
trip_duration_lower_bound = trip_duration_Q1 - 1.5 * trip_duration_IQR
trip_duration_upper_bound = trip_duration_Q3 + 1.5 * trip_duration_IQR

print(f"Soglia inferiore trip_duration: {trip_duration_lower_bound}")
print(f"Soglia superiore trip_duration: {trip_duration_upper_bound}")

# Calcolo dei quartili
total_amount_Q1 = taxi_df['total_amount'].quantile(0.25)
total_amount_Q3 = taxi_df['total_amount'].quantile(0.75)
total_amount_IQR = total_amount_Q3 - total_amount_Q1

# Soglie per identificare gli outlier
total_amount_lower_bound = total_amount_Q1 - 1.5 * total_amount_IQR
total_amount_upper_bound = total_amount_Q3 + 1.5 * total_amount_IQR

print(f"Soglia inferiore total_amount: {total_amount_lower_bound}")
print(f"Soglia superiore total_amount: {total_amount_upper_bound}")

# Calcolo dei quartili
tip_amount_Q1 = taxi_df['tip_amount'].quantile(0.25)
tip_amount_Q3 = taxi_df['tip_amount'].quantile(0.75)
tip_amount_IQR = tip_amount_Q3 - tip_amount_Q1

# Soglie per identificare gli outlier
tip_amount_lower_bound = tip_amount_Q1 - 1.5 * tip_amount_IQR
tip_amount_upper_bound = tip_amount_Q3 + 1.5 * tip_amount_IQR

print(f"Soglia inferiore tip_amount: {tip_amount_lower_bound}")
print(f"Soglia superiore tip_amount: {tip_amount_upper_bound}")

# Rimozione degli outlier
taxi_df = taxi_df[(taxi_df['trip_distance'] <= trip_distance_upper_bound) &
                  (taxi_df['trip_duration'] <= trip_duration_upper_bound)&
                  (taxi_df['total_amount'] <= trip_duration_upper_bound)&
                  (taxi_df['tip_amount'] <= trip_duration_upper_bound)].copy()

num_records = len(taxi_df)
print(f"Number of records in the dataframe: {num_records}")

Number of records in the dataframe: 6201937
Soglia inferiore trip_distance: -1.9500000000000002
Soglia superiore trip_distance: 5.8100000000000005
Soglia inferiore trip_duration: -9.108333333333334
Soglia superiore trip_duration: 32.15833333333333
Soglia inferiore total_amount: -1.4399999999999977
Soglia superiore total_amount: 32.16
Soglia inferiore tip_amount: -4.29
Soglia superiore tip_amount: 7.15
Number of records in the dataframe: 5413365


In [13]:
# Aggiunta della colonna 'day_of_week' derivata dalla data di pickup
taxi_df['day_of_week'] = pd.to_datetime(taxi_df['tpep_pickup_datetime']).dt.day_name()

taxi_df['RatecodeID'] = taxi_df['RatecodeID'].astype('category')
taxi_df.drop(columns=['tpep_pickup_datetime','tpep_dropoff_datetime'], inplace=True)
print(taxi_df.info())
print(taxi_df.iloc[200])

<class 'pandas.core.frame.DataFrame'>
Index: 5413365 entries, 0 to 6339566
Data columns (total 18 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   VendorID               category
 1   passenger_count        float64 
 2   trip_distance          float64 
 3   RatecodeID             category
 4   PULocationID           int64   
 5   DOLocationID           int64   
 6   payment_type           category
 7   fare_amount            float64 
 8   extra                  float64 
 9   mta_tax                float64 
 10  tip_amount             float64 
 11  tolls_amount           float64 
 12  improvement_surcharge  float64 
 13  total_amount           float64 
 14  congestion_surcharge   float64 
 15  trip_duration          float64 
 16  fare_per_mile          float64 
 17  day_of_week            object  
dtypes: category(3), float64(12), int64(2), object(1)
memory usage: 676.3+ MB
None
VendorID                       1.0
passenger_count                1.0

1. **One hot encoding**:

       ```

2. **Normalizzazione**:



In [14]:
taxi_df_cat = pd.get_dummies(taxi_df, columns=['payment_type', 'RatecodeID', 'VendorID','day_of_week'])

#taxi_df_nocat.drop(columns=['PULocationID','DOLocationID'], inplace=True)
print(taxi_df_cat.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5413365 entries, 0 to 6339566
Data columns (total 35 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   passenger_count        float64
 1   trip_distance          float64
 2   PULocationID           int64  
 3   DOLocationID           int64  
 4   fare_amount            float64
 5   extra                  float64
 6   mta_tax                float64
 7   tip_amount             float64
 8   tolls_amount           float64
 9   improvement_surcharge  float64
 10  total_amount           float64
 11  congestion_surcharge   float64
 12  trip_duration          float64
 13  fare_per_mile          float64
 14  payment_type_1.0       bool   
 15  payment_type_2.0       bool   
 16  payment_type_3.0       bool   
 17  payment_type_4.0       bool   
 18  payment_type_5.0       bool   
 19  RatecodeID_1.0         bool   
 20  RatecodeID_2.0         bool   
 21  RatecodeID_3.0         bool   
 22  RatecodeID_4.0         

In [15]:
# Lista delle colonne numeriche
numeric_features = ['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
                    'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge',
                    'total_amount', 'trip_duration', 'fare_per_mile']

# Creiamo una copia del dataframe originale
taxi_df_scaled = taxi_df_cat.copy()

# Applichiamo la normalizzazione solo alle colonne numeriche
scaler = MinMaxScaler()
taxi_df_scaled[numeric_features] = scaler.fit_transform(taxi_df_cat[numeric_features])

# Assicuriamoci che le colonne non numeriche siano preservate
# Nota: Non è necessario rimuovere nuovamente le colonne numeriche.
non_numeric_features = taxi_df_cat.drop(columns=numeric_features).select_dtypes(exclude=['number']).columns

# Combiniamo il dataframe normalizzato con le colonne non numeriche
taxi_df_scaled[non_numeric_features] = taxi_df_cat[non_numeric_features]



In [16]:
print(taxi_df_scaled.info())
print(taxi_df_scaled.iloc[200])

<class 'pandas.core.frame.DataFrame'>
Index: 5413365 entries, 0 to 6339566
Data columns (total 35 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   passenger_count        float64
 1   trip_distance          float64
 2   PULocationID           int64  
 3   DOLocationID           int64  
 4   fare_amount            float64
 5   extra                  float64
 6   mta_tax                float64
 7   tip_amount             float64
 8   tolls_amount           float64
 9   improvement_surcharge  float64
 10  total_amount           float64
 11  congestion_surcharge   float64
 12  trip_duration          float64
 13  fare_per_mile          float64
 14  payment_type_1.0       bool   
 15  payment_type_2.0       bool   
 16  payment_type_3.0       bool   
 17  payment_type_4.0       bool   
 18  payment_type_5.0       bool   
 19  RatecodeID_1.0         bool   
 20  RatecodeID_2.0         bool   
 21  RatecodeID_3.0         bool   
 22  RatecodeID_4.0         

In [17]:
taxi_df_scaled['tipped'] = taxi_df_scaled['tip_amount'].apply(lambda x: 1 if x > 0 else 0)
taxi_df_scaled.drop(columns=['tip_amount'], inplace=True)


plt.figure(figsize=(300,150))
ax = sns.heatmap(taxi_df_scaled[taxi_df_scaled.columns].corr(),cmap="Blues",annot=True,annot_kws={"size": 18})

Output hidden; open in https://colab.research.google.com to view.

In [18]:
# Eliminare tipi di pagamento

#taxi_df_scaled.drop(columns=['payment_type_1.0','payment_type_2.0','payment_type_3.0','payment_type_4.0','payment_type_5.0'], inplace=True)

In [19]:
print(taxi_df_scaled.info())
print(taxi_df_scaled.iloc[200])

<class 'pandas.core.frame.DataFrame'>
Index: 5413365 entries, 0 to 6339566
Data columns (total 30 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   passenger_count        float64
 1   trip_distance          float64
 2   PULocationID           int64  
 3   DOLocationID           int64  
 4   fare_amount            float64
 5   extra                  float64
 6   mta_tax                float64
 7   tolls_amount           float64
 8   improvement_surcharge  float64
 9   total_amount           float64
 10  congestion_surcharge   float64
 11  trip_duration          float64
 12  fare_per_mile          float64
 13  RatecodeID_1.0         bool   
 14  RatecodeID_2.0         bool   
 15  RatecodeID_3.0         bool   
 16  RatecodeID_4.0         bool   
 17  RatecodeID_5.0         bool   
 18  RatecodeID_6.0         bool   
 19  RatecodeID_99.0        bool   
 20  VendorID_1.0           bool   
 21  VendorID_2.0           bool   
 22  day_of_week_Friday     

In [20]:
taxi_df_scaled.to_csv('dataset_classification.csv', index=False)